# Exploring Twitter data
In this notebook, we are going to explore our cleaned dataset that we created in the [first notebook.](1.Data_wrangling_and_cleaning.ipynb)

We are going to do some exploratory analysis in order to understand the shape of the data, patterns and values, correlations between features, and hidden meaning behind our data.

## Goal: 
Learn some common aspects of data exploration, calculate some statistics and visualize data column by column.

## Introduction to exploratory data analysis
[Exploratory data analysis](https://en.wikipedia.org/wiki/Exploratory_data_analysis) is an approach to analyzing data sets to summarize their main characteristics, often with visual methods. (Wikipedia)


### Data exploration process:
 - Computing summary statistics
 - Plotting
 - Additional wrangling if needed.

## Load python modules

There are many packages available in Python that provide a variety of functions for data science. Here we load the libraries needed for this notebook. 

Load the python libraries first. Additional libraries that we are going to use in this notebook are: 
 - [wordcloud](https://amueller.github.io/word_cloud/) -  used to create wordclouds in Python
 - [nltk](https://www.nltk.org/) - natural language toolkit, library to work with language.
 - [folium](https://python-visualization.github.io/folium/) - library for creating maps.
 
We'll also be using [pandas](https://pandas.pydata.org/) to work with DataFrames, one of the most common libraries used for data science. There are good tutorials available on pandas and worthwhile spending more time with in order to become more comfortable with data science in python. 

In [ ]:
import urllib.request

import pandas as pd   #will reference pandas as pd

import nltk
import matplotlib

import matplotlib.pyplot as plt #will reference matplotlib as plt
%matplotlib inline

try:
    from wordcloud import WordCloud
except ImportError:
    !pip install  --user wordcloud
    from importlib import reload
    import site
    reload(site)
    from wordcloud import WordCloud


try:
    import folium
    from folium.plugins import MarkerCluster
except ImportError:
    !pip install  --user folium
    from importlib import reload
    import site
    reload(site)
    import folium
    from folium.plugins import MarkerCluster

#### First, we will read the cleaned dataset from helper git repo and display the first 5 rows

There is a copy of the cleaned dataset (created in the first notebook) downloaded into the helper git repository `data-science-for-entrepreneurs-helper`.  

We will read the csv file into pandas DataFrame and print the first 5 rows.

In [ ]:
file_name="./data/alberta_tweets_cleaned_feb_25.csv"

# reading 'created_at_date' column as timestamp
tweets_cleaned = pd.read_csv(file_name, parse_dates=['created_at_date']) 

#timezone needs to be converted again
tweets_cleaned['created_at_date']=tweets_cleaned['created_at_date'].dt.tz_localize('UTC').dt.tz_convert('MST')

tweets_cleaned.head()

### Short Intro to pandas DataFrames
<details>
<summary> Click here to learn some basic operations on pandas DataFrames </summary>
<br>

`tweets_cleaned` is a DataFrame - 2dimentional data structure(similar to a table). We work with DataFrames in Python using pandas python library.    
Note, there are a few different ways for accessing data inside a DataFrame. We'll cover a couple of aspects here, but you may also want to read more in the pandas documentation([here](https://www.datacamp.com/community/tutorials/pandas-tutorial-dataframe-python) and 
[here](https://cmdlinetips.com/2018/02/how-to-subset-pandas-dataframe-based-on-values-of-a-column/)) or cheat sheet such as the one handed out [here](https://www.dataquest.io/blog/large_files/pandas-cheat-sheet.pdf). 

<h4> Basic operations with DataFrames</h4>

<h5>Display on the screen</h5>

**head()** - without arguments displays first 5 rows, or takes number of rows to display as an argument:    

```python
tweets_cleaned.head()   
tweets_cleaned.head(10)
```

DataFrame name  will display the entire dataframe:   
```python
tweets_cleaned
```

Note, you might need to use **print()** function if there are multiple dataframes displayed in one code cell:  
```python
print(tweets_cleaned.head())
```

<h5>Select set of rows/colums</h5>

Selecting one column:   
```python
tweets_cleaned['created_at_date']
```

Selecting multiple columns:  
```python
tweets_cleaned[['created_at_date','extended_tweet_cleaned']]  
```

Selecting one row(row 2):  
```python
tweets_cleaned.iloc[[2]]
```  

Selecting multiple rows(rows 2 and 5):   
```python
tweets_cleaned.iloc[[2,5]]
``` 

Selecting range of rows(rows 2,3,4 and 5):   
```python
tweets_cleaned.iloc[[2:5]]
```

Selecting rows and columns:   
```python
tweets_cleaned[['created_at_date','extended_tweet_cleaned']].iloc[[2,5]]
```  

Note, row numbers start at 0.
   
<h5>Subset by value in specific column</h5>
 
First we need to create a boolean expression for a subset(let's say we want to filter out rows where value of `user_location` is equal to `Calgary,AB` ):   

```python
bool_expression= tweets_cleaned['user_location']=="Calgary,AB"
```


**Boolean expression**  in this case is an expression that can be either **True** or **False** for every line in DataFrame, for example:   
```python
0        False  
1        False  
2        False  
3        False  
4        True  
5        True  
6        True  
7        False  
8        False  
9        False  
10       True  
...
```
    



Subset DataFrame by boolean expression:   
```python
tweets_cleaned[bool_expression].head(10)
```

**Other  examples of boolean expressions**:

Not equal:   
```python
bool_expression = tweets_cleaned['user_location']!="Calgary,AB"
```

Is part of list of values:
```python
bool_expression = tweets_cleaned['user_location'].isin(["Calgary,AB","Canada"])
```

Multiple conditions  - "and":
```python
bool_expression = (tweets_cleaned['user_location']=="Calgary,AB") & (tweets_cleaned['name']!="Dave")
```

Multiple conditions - "or":  
```python
bool_expression = (tweets_cleaned['user_location']=="Calgary,AB") | (tweets_cleaned['name']!="Dave")
```

## Data Exploration

#### We will explore the data column by column 
A couple of things to keep in mind while doing this is what are we expecting from the data? Are our observations consistent with these expectations? If not, why do they not line up? Are there any trends, outliers, or interesting observations to make note of? 

### `extended_tweet_cleaned` column

Let's begin with this column as this is likely the most interesting one to us. It contains all the text data tweeted out in each tweet. We would like to analyze this further. One way to examine the tweets is by looking at word frequencies. This could be done by using a bar chart or alternatively, using a word cloud. 

As you might imagine, if  we just go ahead and create a word cloud, it will be dominated by some very commonly used words, such as "the", "or", "and", etc. In order to prevent these common words from dominating the plot, we will remove them. Commonly used words that don't add anything contextually interesting such as this are called "stopwords". The wordcloud package that we will be using has a pre-built list of stopwords. 

Let's import them and examine:

In [ ]:
from wordcloud import  STOPWORDS
print(STOPWORDS)

Now we'll use the [WordCloud()](https://amueller.github.io/word_cloud/generated/wordcloud.WordCloud.html#wordcloud.WordCloud) function from the wordcloud library.

In [ ]:
stopwords = set(STOPWORDS)

#In the following function, random_state is set for reproducibility and collocation=False means 
#that if the same word is encountered in collocation with different words,
#it is still considered as one word (to remove duplicates)
def wordcloud(tweets,col):
    words=" ".join([i for i in tweets[col]]) # we join all tweets in one character string 
    wordcloud = WordCloud(width=800, height=400,collocations=False,background_color="white",stopwords=stopwords,random_state = 2019).generate(words)
    plt.figure( figsize=(20,10))
    plt.imshow(wordcloud)
    plt.axis("off") 

In [ ]:
wordcloud(tweets_cleaned,'extended_tweet_cleaned')


All these words make a lot of sense, except for... amp?? Let's find some sample tweets that will allows us to inspect them why people might be tweeting about amps. 

In [ ]:
pd.set_option('max_colwidth', 700)
rows_containing_amp=tweets_cleaned['extended_tweet_cleaned'].str.contains("amp")
tweets_cleaned[rows_containing_amp]['extended_tweet_cleaned'].head(10)

It appears that &amp is just a shortcut for ampersand. This is a bit misleading and so let's delete all the &amp occurences from the 'extended_tweet_cleaned' column. 

What would be another way of dealing with the confusing &amp text? 

In [ ]:
tweets_cleaned['extended_tweet_cleaned']=tweets_cleaned['extended_tweet_cleaned'].str.replace('&amp',' ')

#### Exclude "one" and "will" words
"one" and "will" words can possibly be excluded as well, they don't have any special meaning. In this case, we'll add them to the list of stop words.

In [ ]:
stopwords.add("one")
stopwords.add("will")
wordcloud(tweets_cleaned,'extended_tweet_cleaned')

#### Exercise1: add Calgary and AB (or other words as you like) to stopwords to exclude them from the wordcloud and try plotting the wordcloud again.

In [ ]:
### type something here


###
wordcloud(tweets_cleaned,'extended_tweet_cleaned')

OK - this word cloud looks more sensible. What picture emerges from this? Are there any  themes we can identify? People seem to be thankful, something about time - maybe mentioning a good time? While we can guess at some of these aspects, it's impossible to say anything about the context these words occur in. For that, we would need some more sophisticated analyses. This could include aspects like n-gram analysis or topic modelling, which is what we will take a look at in the next notebook. 

### `Created_at_date` column
This column is of type Timestamp. 
The following [features](https://www.geeksforgeeks.org/python-working-with-date-and-time-using-pandas/) can be useful for working with the Timestamp format:
 - **dt.year** returns the year of the date time.
 - **dt.month** returns the month of the date time.
 - **dt.day** returns the day of the date time.
 - **dt.hour** returns the hour of the date time.
 - **dt.minute** returns the minute of the date time.
   
 **min(), max()** functions can be used with timestamp as well.   
   
 Let's find out the time range first:

In [ ]:
print("Time range: ",min(tweets_cleaned["created_at_date"]),"-",max(tweets_cleaned["created_at_date"]))

We have data from February 25th to March 4th.
#### Let's group by day and calculate the number of tweets for each day.

We will use the [groupby](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html) function, which is a pandas function that groups rows into groups based on one or multiple columns.

In [ ]:
##size of the plot is 5/5 square to make it circle, not oval 
plt.figure(figsize=(5,5))

#group by day
tweets_cleaned_by_day=tweets_cleaned.groupby(tweets_cleaned["created_at_date"].dt.day)

#Select the 'created_at_date' column from grouped dataframe count up, and plot
tweets_cleaned_by_day["created_at_date"].count().plot(kind="pie")

This pie chart shows the proportion of tweets per day. The number on the circle corresponds to the day of the month the data was collected. 

Another way to plot this is as a bar chart: 

In [ ]:
plt.figure(figsize=(10,5)) ## making bigger plot rectangular shape in this case

ax = tweets_cleaned_by_day["created_at_date"].count().plot(kind="bar")

#Note in the line above that all we changed from the pie chart is the type of graph we requested
ax.set_xlabel("Day", size =16)
ax.set_ylabel("Counts", size=16)
plt.show()

It looks like the most tweets took place on March 2nd. Let's confirm that and check out some additional summary stats. 

In [ ]:
#shape[0] - gives number of rows, shape[1] - number of columns

print("Total number of tweets: ",
      tweets_cleaned.shape[0])

#subsetting only rows where day is equal to 2
bool_expression_day_2 = tweets_cleaned["created_at_date"].dt.day==2

print("Number of tweets collected on March 2nd: ", 
      tweets_cleaned[bool_expression_day_2].shape[0])


#### Let's take a look at the number of tweets by hour of day and whether this make sense with what we expect. What kind of pattern would you expect? 
 
#### Exercise2: Using the same methodology as for the bar chart above, can you plot the number of tweets by hour? 
Note this is a summary plot for the same hour for all days.

In [ ]:
## type something here


######################

In [ ]:
#group by hour
tweets_cleaned_by_hour=tweets_cleaned.groupby(tweets_cleaned["created_at_date"].dt.hour)

print("Total number of tweets collected between 03:00 and 04:00: ", 
      tweets_cleaned_by_hour["created_at_date"].count()[3])

print("Total number of tweets collected between 18:00 and 19:00: ", 
      tweets_cleaned_by_hour["created_at_date"].count()[18])


#### Exercise3: plot tweets number by day and hour 
**Hint1**:   
For two values in groupby() function:
>instead of `groupby([value])` use `groupby([value1,value2])` 

Note how plot can be different if you do `groupby([day,hour])`  vs `groupby([hour,day])`.

**Hint2**: 
>Use `figsize=(20,10)` to make larger plot

**Optional**:  
Use this code snippet to display every 4th x axis label, to make plot more clear:
>``` python
  for n, label in enumerate(ax.xaxis.get_ticklabels()):
>     if n % 4 != 0:
>        label.set_visible(False)
```

In [ ]:
## type something here



######################

OK - so far everything looks as we expected it for when the tweets were collected and at which time of day users were  most active. Let's move on to the next column. 

### `Hashtag_string`  and `user_string` columns
These two columns are of type String and have a list of hastags/user mentions separated by a blank.  
We will create a list of all the hashtags first using the [join()](https://www.tutorialspoint.com/python/string_join.htm) and [split()](https://www.w3schools.com/python/ref_string_split.asp) functions. 

In [ ]:
all_hashtags=" ".join([i for i in tweets_cleaned['hashtags_string']]) #join all the hashtags into one character sting
all_hashtags=all_hashtags.split()  # transform into a list
print(all_hashtags)

Then we use the [FreqDist()](https://kite.com/python/docs/nltk.probability.FreqDist) function from the nltk library to get frequency distributions for all the words.  That is, this function will count how many times each hashtag occurs. 

In [ ]:
freque_dist=nltk.FreqDist(all_hashtags)
print("Most common hashtags: ",freque_dist.most_common(20)) ## most_common(n) function prints top n words with highest frequencies

We can also visualize this using matplotlib to print the top 25 most common hashtags. It's possible to use the [plot()](https://kite.com/python/docs/nltk.probability.FreqDist.plot) function directly with a  FreqDist object.

In [ ]:
plt.figure(figsize=(10,5)) # plot size
plt.title("Most common hashtags", size = 20)
plt.xlabel("Hashtag", size = 16)
plt.ylabel("Counts", size = 16)
plt.xticks(fontsize=14) ## Change font for x axis labels
freque_dist.plot(25)

We will do the same steps for the user_string column:

In [ ]:
all_users=" ".join([i for i in tweets_cleaned['user_string']])
all_users=all_users.split()


plt.figure(figsize=(10,5))
plt.title("Most common user mentions", size=20)
plt.xlabel("Tagged User", size=16)
plt.ylabel("Number of tags", size =16)
plt.xticks(fontsize=14)
fd = nltk.FreqDist(all_users)
fd.plot(25)

#### How many users talked about the most commonly mentioned user? 
We will use the [str.contains()](https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.Series.str.contains.html) function to get rows containing specific string:
> tweets_cleaned['user_string'].str.contains("JustinTrudeau")

In [ ]:
bool_expression_JT=tweets_cleaned['user_string'].str.contains("JustinTrudeau")

print("Number of tweets with the most common user mention 'JustinTrudeau':",
      len(tweets_cleaned[bool_expression_JT]['name']))


print("Number of users using the most common user mention 'JustinTrudeau':",
      len(tweets_cleaned[bool_expression_JT]['name'].unique()))

By contrast, how does this compare to the second most commonly mentioned user? Check the bar chart above and find out how many users mentioned it. 


In [ ]:
bool_expression_BTS=tweets_cleaned['user_string'].str.contains("BTS_twt")

print("Number of tweets with the most common user mention 'BTS_twt':",
      len(tweets_cleaned[bool_expression_BTS]['name']))

print("Number of users using the most common user mention 'BTS_twt':",
      len(tweets_cleaned[bool_expression_BTS]['name'].unique()))

#### Which users used the most number of user mentions?

In [ ]:
max_user_mentions=tweets_cleaned['user_string'].str.split().str.len().max()

rows_with_max_user_mentions=tweets_cleaned['user_string'].str.split().str.len()==max_user_mentions

print("Maximum number of user mentions:",
      max_user_mentions,
      "made by  ",
      tweets_cleaned[rows_with_max_user_mentions] ['name'].unique())

#### Exercise4 - try subsetting the data by hour (for example between 9 and 10 am) and plot the most common hashtags or user_mentions
Hint: to select only data between 9 and 10 am use the dt.hour==9 boolean expression:
>bool_expression_hour_9=tweets_cleaned["created_at_date"].dt.hour==9   
>tweets_cleaned[bool_expression_hour_9]

In [ ]:
## type something here


######################

### `User_location` column

In [ ]:
unique_locations=tweets_cleaned["user_location"].unique()

print("Number of unique user locations:", len(unique_locations), "\n")
print(unique_locations)

There are 1000 unique user_locations. These are locations that are entered by users in the profile and open to pretty much any input, which is why we see some odd locations in there. Not sure if this is overly interesting right now. Let's look into latitude/longitude instead.

### `Latitude/longitude` columns
We will subset data by day (to make it faster) and plot tweets that have coordinates on a map.

How are latitude/longitude added to a tweet? Do all tweets have them? How reliable is this data source? A few things to keep in mind as we go through this...

In [ ]:
print("Total number of tweets: ",tweets_cleaned.shape[0])

rows_with_location=tweets_cleaned["longitude"].notnull()
tweets_have_location=tweets_cleaned[rows_with_location]

print("Number of tweets having location data: ",tweets_have_location.shape[0])

In [ ]:
rows_have_location_day_2=tweets_have_location["created_at_date"].dt.day==2

tweets_subset_march2=tweets_have_location.loc[rows_have_location_day_2]
print("Number of tweets having location data for March 2: ",tweets_subset_march2.shape[0])

We can plot the location data on a map and use the library [folium](https://github.com/python-visualization/folium). We will iterate through each row in the December 18 subset of data and add the coordinates to the map where they exist. 

This can be accomplished using the [iterrrows()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.iterrows.html) function to iterate through dataframe rows:

In [ ]:
calgary_coords = [51.0486, -114.0708] # Calgary coordinates, (where we are going to center the map)
my_map = folium.Map(location = calgary_coords, zoom_start = 13)

for index,row in tweets_subset_march2.iterrows(): 
        folium.Marker([row["longitude"], row["latitude"]]).add_to(my_map) 
my_map

It looks like these locations are in line with what we expected. The highest distribution falls within the downtown core and if you scroll out, you'll notice that the scraper seems to have done a good job limiting the collected tweets to roughly Alberta.

#### Exercise5: try subsetting by user mention (e.g "calgarylibrary") and  plot tweets on a map
Hint: use str.contains() function applied to tweets_have_location['user_string'] colum:
>bool_expression_usermentions=tweets_have_location['user_string'].str.contains("your_search_word")
>tweets_have_location[bool_expression_usermentions]

In [ ]:
### type something here


######################

### `Screen_name` column

Let's look at some summary stats.

In [ ]:
print("Total number of tweets: ",tweets_cleaned.shape[0])
print("Total number of users: ",tweets_cleaned['screen_name'].unique().shape[0])
print("Top 20 most active users:")
tweets_grouped_by_screenname=tweets_cleaned["screen_name"].groupby(tweets_cleaned["screen_name"])
tweets_grouped_by_screenname.count().reset_index(name='count').sort_values(['count'], ascending=False).head(20)#.plot(kind="bar")

### Save data to csv
We have done a whole lot of exploring and some additonal data wrangling that has changed the data, so let's save an updated dataset to csv.

In [ ]:
tweets_cleaned.to_csv('alberta_tweets_cleaned_feb_25_updated.csv',index=False)

##  Conclusion

We've done some data exploration in order to try to understand the data better. Our time spent on each column depended on how complex the data is each column. Since we will continue to work with the text data, that is where we started and spent most of our time working with. 

Take-aways: 
* Data exploration helps provide a sense of the data that is there and often reveals interesting trends and patterns 
* Always consider both exploring the data using summary statistics along with data visualizations
* Think about what you would expect the data to look like going into it and determine whether your assumptions hold up. If not, why not? This could tell you important things about your data. 
* Subject matter expertise is invaluable in data science, including in the exploration phase. A solid understanding of how Twitter is used, or working with someone who does, can often quickly resolve questions that could otherwise take hours to answer. 

Next, we will go deeper into natural language processing, will  build topic models  and do sentiment analysis

## Solutions

#### Exercise1:

In [ ]:
stopwords.add("Calgary")
stopwords.add("Alberta")

#### Exercise2:

In [ ]:
#group by hour
tweets_cleaned_by_hour=tweets_cleaned.groupby(tweets_cleaned["created_at_date"].dt.hour)

#Select the 'created_at_date' column from grouped dataframe count up, and plot
ax = tweets_cleaned_by_hour["created_at_date"].count().plot(kind="bar", figsize=(10,5))
ax.set_xlabel("Hour", size =16)
ax.set_ylabel("Counts", size=16)
plt.show()

#### Exercise3:

In [ ]:
ax = tweets_cleaned["created_at_date"].groupby([tweets_cleaned["created_at_date"].dt.day,tweets_cleaned["created_at_date"].dt.hour]).count().plot(kind="bar", figsize=(20,10))
ax.set_xlabel("Day/Hour", size =16)
ax.set_ylabel("Counts", size=16)
##To display every 4th tick for x axis:
for n, label in enumerate(ax.xaxis.get_ticklabels()):
    if n % 4 != 0:
        label.set_visible(False)
plt.show()

#### Exercise4:

In [ ]:
bool_expression_hour_9=tweets_cleaned["created_at_date"].dt.hour==9
tweets_cleaned_9=tweets_cleaned[bool_expression_hour_9]

all_users=" ".join([i for i in tweets_cleaned_9['user_string']])
all_users=all_users.split()

plt.figure(figsize=(10,5))
plt.title("Most common user mentions between 9 and 10 am", size=20)
plt.xlabel("Tagged User", size=16)
plt.ylabel("Number of tags", size =16)
plt.xticks(fontsize=14)
fd = nltk.FreqDist(all_users)
fd.plot(25)

#### Exercise5:

In [ ]:
bool_expression_usermentions=tweets_have_location['user_string'].str.contains("calgarylibrary") 
tweets_subset_library=tweets_have_location[bool_expression_usermentions]

my_map = folium.Map(location = calgary_coords, zoom_start = 13)

for index,row in tweets_subset_library.iterrows():  
        folium.Marker([row["longitude"], row["latitude"]]).add_to(my_map) 
my_map